In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/facial_expression_detection/research'

In [2]:
os.chdir('../')
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/facial_expression_detection'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path

In [4]:
from src.detmood.constant import *
from src.detmood.utils.main_utils import create_directories, read_yaml

class ConfigurationMananger:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
    
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file
        )
        
        return data_ingestion_config

In [7]:
import os
from src.detmood.utils.main_utils import get_size, create_directories
from src.detmood import logger
import zipfile
import subprocess

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_dataset(self):
        if not os.path.exists(self.config.local_data_file):
            info = subprocess.run(
                f"wget '{self.config.source_URL}' -O {self.config.local_data_file}",
                shell=True
            )
            logger.info(f"Dataset downloaded with folowing info: \n{info}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def download_kaggle_dataset(self):
        if not os.path.exists(self.config.local_data_file):
            info = subprocess.run(
                f"kaggle datasets download -d {self.config.source_URL} -p {self.config.root_dir}",
                shell=True
            )
            logger.info(f"Zip file of dataset downloaded with folowing info: \n{info}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.root_dir)
        logger.info("Zip file extacted!")
        
        os.remove(self.config.local_data_file)
        logger.info("Zip file removed!")
    
    def ingestion_compose(self):
        # if not os.listdir(self.config.root_dir):
        if not os.path.exists(self.config.local_data_file):
            # self.download_dataset()
            self.download_kaggle_dataset()
            self.extract_zip_file()
        else:
            print("Data ingestion allready performed!")

In [8]:
try:
    config = ConfigurationMananger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.ingestion_compose()

except Exception as e:
    raise e

[2024-11-27 17:16:04,120: INFO: main_utils: created directory at: artifacts]
[2024-11-27 17:16:04,122: INFO: main_utils: created directory at: artifacts/data_ingestion]


Dataset URL: https://www.kaggle.com/datasets/shuvoalok/raf-db-dataset
License(s): other


 90%|█████████ | 34.0M/37.7M [00:01<00:00, 26.3MB/s]


[2024-11-27 17:16:07,635: INFO: 1841096850: Zip file of dataset downloaded with folowing info: 
CompletedProcess(args='kaggle datasets download -d shuvoalok/raf-db-dataset -p artifacts/data_ingestion', returncode=0)]


100%|██████████| 37.7M/37.7M [00:02<00:00, 19.2MB/s]


[2024-11-27 17:16:09,685: INFO: 1841096850: Zip file extacted!]
[2024-11-27 17:16:09,692: INFO: 1841096850: Zip file removed!]
